In [1]:
from kernels import * 
from learning_models import *
from tools import *
import pandas as pd
import numpy as np
from time import time 
from sklearn.svm import SVC
from autoreload import superreload

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataX = pd.read_csv("Data/Train/Xtr0.csv", header=None)

In [4]:
dataX.head()

,0
0,TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGG...
1,CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCA...
2,GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGT...
3,TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTC...
4,CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCT...


In [5]:
data = dataX[0].values.tolist()

In [6]:
data[:10]

['TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG',
 'CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCACAGGGCAAAGACTTCTCAGAGACAACTAATATGGCCTAGTGCCTAGTCATTGGAC',
 'GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGTGCGTGTCTGGCTGCAGTTGATAGTTGGGCAAAGAGGAAACTATCACAAAGAGGAA',
 'TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTCACAGACTTCTCTGTGCTCCCATCTAAACATCTAAATCAGCCATCTGGTTTCCTTA',
 'CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCTAGAGTGTCTGCCATGTCTATCAATTGGATTACGGGGCTTTTATTGATTAGGGGGA',
 'GAAACAAGCTAATATAAGGTGGGGGCCTGAAAATTTTATCCAATGGTCATTTAAGAATCGCAATCAATGTCCTCTAGATATATAATTCACCTAAATACGGA',
 'CTTTCTTATCTGTGTGGCTGTGGGCGTGTCTTTGGCAAGCCCCACTGTGAGAGTTCCCTTATCTATGCCTGCAGGCTGTTCTTTTATTTGAAAGAATTCAG',
 'TGATTACTGCATGGAAAAGGGGCCAGGGCAATTACTAGAGGGCTCAATCCCAGTAAGGGGATGTGTACCAGGGAGGGTTGCTTTGACAGGAGATCAGTCTA',
 'GAGATTCTTAACATGCAACCACAAATCGATGCAGTTATGGATCATGTTGAATGGCAACTCACCTGGCAGTGGAGAGCAGAGTTGTTGGCAATCACATATGA',
 'GGGGACTTCTTGAACTGAGCTCCTAGAGCAGCTGGGGTTGTGAT

In [7]:
d = []
for i in range(len(data)):
    s = data[i].replace("A", "0")
    s = s.replace("T", "1")
    s = s.replace("C", "2")
    s = s.replace("G", "3")
    d.append(list(map(int, s)))
data = d

In [8]:
del dataX

In [11]:
trie = MismatchTrie(verbose=0)
print("-------- Computing mismatch kernel --------")
start = time()
kern = trie.traverse(data, 4, 5, 1)[0]
#kern = normalize_kernel(kern)
end = time()
print("-------- Done in {:.3f} minutes --------".format((end-start)/60))

-------- Computing mismatch kernel --------
-------- Done in 90.561 minutes --------


In [13]:
kern.shape

(2000, 2000)

In [14]:
kern_data = pd.DataFrame(kern)

In [15]:
kern_data.to_csv('Kernel_MisMatch_4_1.csv')

In [33]:
np.array(data).shape

(2000, 101)

In [16]:
dataY = pd.read_csv("Data/Train/Ytr0.csv", index_col=0)
y = dataY['Bound'].values
X = np.array(data)
K = kern

In [17]:
kern = normalize_kernel(kern)

In [217]:
X_train, X_test, y_train, y_test, K_train, K_test = train_test_split(X,y,K,test_size=0.1)

Total number of examples : 2000
Ratio of positive samples : 0.50
Ratio of negative to positive labels in the data : 1.00


In [45]:
X_train, X_test, y_train, y_test, K_train, K_test = K_train_test_split(X,y,K,test_size=0.2)

In [218]:
res = []
for lmdb in [0.001,0.01,0.05,0.1,1,10,100,1000]:
    model = SVM(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    train_score = accuracy_score(y_train, y_train_pred)
    test_score = accuracy_score(y_test, y_test_pred)
    print("\n\n------ Lambda = {} ------".format(lmdb))
    print('Train accuracy score = {:.3f}'.format(train_score))
    print('Test accuracy score = {:.3f}\n\n'.format(test_score))
    res.append([lmdb,test_score])


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -3.7890e+00 -4.4986e+00  1e+02  2e+00  3e-02
 2: -2.2564e+00 -2.2930e+00  2e+01  3e-01  5e-03
 3: -9.2195e-01 -1.7995e+00  3e+00  3e-02  5e-04
 4: -7.8162e-01 -1.1861e+00  4e-01  6e-18  5e-15
 5: -8.8444e-01 -9.3520e-01  5e-02  4e-18  4e-15
 6: -9.0482e-01 -9.0686e-01  2e-03  5e-18  4e-15
 7: -9.0570e-01 -9.0590e-01  2e-04  5e-18  4e-15
 8: -9.0579e-01 -9.0581e-01  2e-05  5e-18  4e-15
 9: -9.0580e-01 -9.0580e-01  2e-07  6e-18  4e-15
Optimal solution found.


------ Lambda = 0.001 ------
Train accuracy score = 0.753
Test accuracy score = 0.535



			######## y has been reformated to {-1,1} ########

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -2.7299e+01 -1.7740e+01  5e+02  7e+00  1e-01
 2: -8.7977e+00 -3.7161e+00  8e+01  1e+00  2e-02
 3: -1.5724e+00 -1.9576e+00  8e+00  9e-02  2e-03
 4: -8.0822e-01 -1.6184e+00  8e-01  1e-17  2e-15


In [219]:
best_pred = sorted(res, key = lambda x:x[1])[-1]
print(best_pred)

[1000, 0.535]


In [27]:
print(lmbd)

100000


In [156]:
lmbd = 100000
model = SVM(lmbd=lmbd)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred = model.predict(K_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)
print("\n\n------ Lambda = {} ------".format(lmbd))
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n\n'.format(test_score))


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -9.9146e-01 -1.7627e+00  5e+01  6e-01  1e-02
 2: -7.1546e-01 -1.7040e+00  1e+00  6e-17  2e-15
 3: -9.2579e-01 -1.0071e+00  8e-02  7e-18  2e-15
 4: -9.9926e-01 -1.0001e+00  9e-04  5e-18  2e-15
 5: -9.9999e-01 -1.0000e+00  9e-06  6e-18  2e-15
 6: -1.0000e+00 -1.0000e+00  9e-08  5e-18  2e-15
Optimal solution found.


------ Lambda = 100000 ------
Train accuracy score = 0.761
Test accuracy score = 0.580




In [53]:
lmbd = 5e-4
model = SVM(lmbd=lmbd)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred = model.predict(K_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)
print("\n\n------ Lambda = {} ------".format(lmbd))
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n\n'.format(test_score))


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  3e+03  6e+01  1e+00
 1: -2.4207e+00 -3.2786e+00  1e+02  2e+00  3e-02
 2: -1.5022e+00 -2.0293e+00  1e+01  2e-01  4e-03
 3: -7.3850e-01 -1.6413e+00  2e+00  1e-02  2e-04
 4: -7.2831e-01 -9.8523e-01  3e-01  2e-03  3e-05
 5: -7.7176e-01 -8.6485e-01  1e-01  5e-04  8e-06
 6: -7.9458e-01 -8.1709e-01  2e-02  6e-05  1e-06
 7: -8.0247e-01 -8.0420e-01  2e-03  1e-07  2e-09
 8: -8.0322e-01 -8.0332e-01  1e-04  7e-09  1e-10
 9: -8.0326e-01 -8.0326e-01  3e-06  2e-10  3e-12
10: -8.0326e-01 -8.0326e-01  7e-08  2e-12  4e-14
Optimal solution found.


------ Lambda = 0.0005 ------
Train accuracy score = 0.787
Test accuracy score = 0.570




In [39]:
X_train

array([[3, 0, 1, ..., 2, 0, 3],
       [0, 0, 1, ..., 2, 0, 1],
       [3, 0, 3, ..., 0, 3, 1],
       ...,
       [3, 1, 1, ..., 3, 2, 1],
       [0, 3, 2, ..., 3, 0, 0],
       [1, 1, 3, ..., 3, 1, 0]])

In [40]:
X_test

array([[3, 3, 2, ..., 1, 3, 2],
       [1, 0, 0, ..., 0, 0, 2],
       [0, 2, 0, ..., 3, 3, 1],
       ...,
       [3, 0, 2, ..., 1, 0, 1],
       [3, 1, 2, ..., 3, 3, 3],
       [2, 1, 3, ..., 1, 0, 2]])

In [58]:
def transform(y, threshold=0.5):
    res = np.zeros(len(y))
    ones = y >= threshold
    res[ones] = 1
    res[~ones] = 0
    return res

In [ ]:
res = []
for lmdb in [1e-5, 1e-4, 0.001,0.01,0.05]:
    print("------ Lambda = {} ------".format(lmdb))
    model = KRR(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        print('\t\tthreshold = {}'.format(threshold))
        res_train = transform(y_train_pred, threshold=threshold)
        res_test = transform(y_test_pred, threshold=threshold)
        train_score = accuracy_score(y_train, res_train)
        test_score = accuracy_score(y_test, res_test)
        print('Train accuracy score = {:.3f}'.format(train_score))
        print('Test accuracy score = {:.3f}\n'.format(test_score))
        res.append([lmdb, threshold, test_score])

In [221]:
best_pred = sorted(res, key = lambda x:x[2])[-1]
print(best_pred)

[0.0001, 0.2, 0.595]


In [200]:
res = []
for lmdb in [0.1, 0.5, 0.1, 5, 10]:
    print("------ Lambda = {} ------".format(lmdb))
    model = KRR(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        print('\t\tthreshold = {}'.format(threshold))
        res_train = transform(y_train_pred, threshold=threshold)
        res_test = transform(y_test_pred, threshold=threshold)
        train_score = accuracy_score(y_train, res_train)
        test_score = accuracy_score(y_test, res_test)
        print('Train accuracy score = {:.3f}'.format(train_score))
        print('Test accuracy score = {:.3f}\n'.format(test_score))
        res.append([lmdb, threshold, test_score])

------ Lambda = 0.1 ------
		threshold = 0.1
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.2
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.3
Train accuracy score = 0.501
Test accuracy score = 0.495

		threshold = 0.4
Train accuracy score = 0.526
Test accuracy score = 0.540

		threshold = 0.5
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.6
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.7
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.8
Train accuracy score = 0.500
Test accuracy score = 0.500

------ Lambda = 0.5 ------
		threshold = 0.1
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.2
Train accuracy score = 0.500
Test accuracy score = 0.490

		threshold = 0.3
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.4
Train accuracy score = 0.500
Test accuracy score = 0.500

		threshold = 0.5
Train accuracy s

In [201]:
best_pred = sorted(res, key = lambda x:x[2])[-1]
print(best_pred)

[0.1, 0.4, 0.54]


In [63]:
int(0.5)

0

In [64]:
int(1.2)

1

In [233]:
model = KRR(lmbd=0.0001)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred_krr = model.predict(K_test)
res_train = transform(y_train_pred, threshold=0.2)
res_test = transform(y_test_pred_krr, threshold=0.2)
train_score = accuracy_score(y_train, res_train)
test_score = accuracy_score(y_test, res_test)
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n'.format(test_score))
res.append([lmdb, threshold, test_score])

Train accuracy score = 0.679
Test accuracy score = 0.595



In [234]:
lmbd = 1000
model = SVM(lmbd=lmbd)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred_svm = model.predict(K_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred_svm)
print("\n\n------ Lambda = {} ------".format(lmbd))
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n\n'.format(test_score))


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -1.1402e+00 -1.7643e+00  5e+01  6e-01  1e-02
 2: -7.1800e-01 -1.7064e+00  1e+00  7e-17  2e-15
 3: -9.2553e-01 -1.0103e+00  8e-02  7e-18  2e-15
 4: -9.9925e-01 -1.0002e+00  9e-04  5e-18  2e-15
 5: -9.9999e-01 -1.0000e+00  9e-06  6e-18  2e-15
 6: -1.0000e+00 -1.0000e+00  9e-08  5e-18  2e-15
Optimal solution found.


------ Lambda = 1000 ------
Train accuracy score = 0.753
Test accuracy score = 0.535




In [69]:
y_test

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1])

In [235]:
(y_test_pred_svm == 1) * (res_test == 0)

array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
        True, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,

In [76]:
transform(y_test_pred, threshold=0.5)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [236]:
on_target = res_test == y_test
one_indices = y_test == 1.
TP = np.sum(on_target * one_indices)
FN = np.sum(~on_target * one_indices)
TN = np.sum(on_target * ~one_indices)
FP = np.sum(~on_target * ~one_indices)

In [228]:
on_target = y_test_pred_svm == y_test
one_indices = y_test == 1.
TP = np.sum(on_target * one_indices)
FN = np.sum(~on_target * one_indices)
TN = np.sum(on_target * ~one_indices)
FP = np.sum(~on_target * ~one_indices)

In [237]:
TP

88

In [238]:
FN

12

In [239]:
TN

31

In [240]:
FP

69

In [93]:
TP / (TP+FN)

0.74

In [94]:
TN / (TN + FP)

0.49

In [97]:
TP + FP

125

In [98]:
len(y_test)

200

In [229]:
TP

56

In [230]:
FN

44

In [231]:
TN

50

In [232]:
FP

50

In [84]:
TP / (TP+FN)

0.62

In [85]:
TN / (TN + FP)

0.71

In [86]:
(0.71 + 0.62)/2

0.665

In [241]:
y_test_pred_combined = res_test.copy()
for i in range(len(y_test)):
    if res_test[i] == 0 and y_test_pred_svm[i] == 1:
        y_test_pred_combined[i] = 1

In [242]:
accuracy_score(y_test_pred_combined, y_test)

0.59

In [187]:
res_test

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])

In [188]:
y_test_pred_combined

array([0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])

In [189]:
y_test_pred_combined == res_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [190]:
(res_test == 0) * (y_test_pred_svm == 1)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

### Regarder si les outputs du KRR correspondent vraiment à des probas : si les outputs proches de 0 sont effectivement mieux classifiés que les outputs 0.4 ...

In [130]:
transform(y_test_pred_krr[:20]).astype(int)

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1])

In [131]:
y_test_pred_svm[:20].astype(int)

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1])

In [129]:
y_test[:20]

array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0])

### Regarder quand les deux rendent 0 combien de fois c'est bon, puis quand l'un rend 0 et l'autre 1 lequel a raison, puis l'inverse, puis quand les deux rendent 1...

In [136]:
res_test_pred_krr = transform(y_test_pred_krr, 0.5)

In [137]:
both_on_target = (y_test_pred_svm == y_test) * (res_test_pred_krr == y_test)

In [138]:
both_on_target

array([ True,  True,  True, False, False, False, False,  True,  True,
        True,  True, False,  True,  True, False, False, False,  True,
       False, False,  True, False, False, False,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False, False, False,  True,  True,  True, False, False, False,
        True,  True,  True,  True, False, False,  True, False, False,
        True, False,  True, False, False,  True, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True, False,  True, False, False,  True,
       False, False,  True,  True, False,  True, False, False, False,
       False, False,  True,  True,  True, False,  True, False, False,
       False,  True,  True,  True,  True,  True,  True, False, False,
        True,  True, False,  True,  True,  True, False,  True, False,
       False,  True, False, False, False,  True,  True,  True, False,
       False, False,

In [139]:
krr_no_svm = (res_test_pred_krr == y_test) * ~(y_test_pred_svm == y_test)

In [140]:
svm_no_krr = (y_test_pred_svm == y_test) * ~(res_test_pred_krr == y_test)

In [141]:
no_svm_no_krr = ~(y_test_pred_svm == y_test) * ~(res_test_pred_krr == y_test)

In [142]:
np.sum(both_on_target)

107

In [143]:
np.sum(svm_no_krr)

16

In [144]:
np.sum(krr_no_svm)

26

In [145]:
np.sum(no_svm_no_krr)

51

In [146]:
107 / 158

0.6772151898734177

In [147]:
svm_0_krr_1 = (y_test_pred_svm == 0) * (res_test_pred_krr == 1)

In [148]:
svm_1_krr_0 = (y_test_pred_svm == 1) * (res_test_pred_krr == 0)

In [149]:
np.sum(svm_0_krr_1)

4

In [150]:
np.sum(svm_1_krr_0)

38

In [151]:
np.sum(svm_0_krr_1 * svm_no_krr)

2

In [152]:
np.sum(svm_1_krr_0 * svm_no_krr)

14

In [ ]:
np.sum(svm_0_krr_1)

In [135]:
y_test_pred_krr == y_test

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,